<a href="https://colab.research.google.com/github/kumar4372/sentiment_analysis_hands_on/blob/master/(trainer)_Using_RNN_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis Using Recurrent Neural Network**

---



## choose Hardware accelarator to "GPU" for faster computation. Go to "Runtime" -> "Change runtime type" to change it.


In this tutorial, we will use RNN for sentiment analysis task on movie review dataset.

**What is sentiment analysis?**

Sentiment Analysis is nothing but finding the sentiments of reviews whether it is positive or negative review.

**Example Code to refer**: https://slundberg.github.io/shap/notebooks/deep_explainer/Keras%20LSTM%20for%20IMDB%20Sentiment%20Classification.html

**Notes**
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.

**Importing Libraries**

We start by importing the required dependencies to preprocess our data and build our model.

In [ ]:
# Import the dependencies
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import Dense, SimpleRNN,LSTM, GRU
from keras.preprocessing import sequence

print("Imported dependencies.")


Imported dependencies.


**Loading Data**

We will use IMDB sentiment classification dataset which consists of 50,000 movie reviews from IMDB users that are labeled as either positive (1) or negative (0). 

Continue downloading the IMDB dataset, which is, fortunately, already built into Keras.

In [ ]:
vocab_size = 10000

# Define the training and test dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)  # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

print("Created test and training data.")
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17473536/17464789 [==============================] - 1s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


Created test and training data.
25000 train sequences
25000 test sequences


/usr/local/lib/python3.7/dist-packages/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


**Exploring the data**

You can see in the output above that the dataset is labeled into two categories, — 0 or 1, which represents the sentiment of the review. The whole dataset contains 9,998 unique words and the average review length is 234 words, with a standard deviation of 173 words.

In [ ]:
import numpy as np

#concatenate whole data
data = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Categories: [0 1]
Number of unique words: 9998
Average Review length: 234.75892
Standard Deviation: 173.0


You can see the first review of the dataset, which is labeled as positive (1). 

In [ ]:
print('---review---')
print(x_train[0])
print('---label---')
print(y_train[0])

---review---
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
---label---
1


Above you can see the first review of the dataset, which is labeled as positive (1). The code below retrieves the dictionary mapping word indices back into the original words so that we can read them. It replaces every unknown word with a “#”. It does this by using the get_word_index() function.

In [ ]:
index = imdb.get_word_index()
train_text = []
test_text = []
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
for i in range(0,len(x_train)):
  train_text.append(" ".join( [reverse_index.get(i - 3, "#") for i in x_train[i]] ))
for i in range(0,len(x_test)):
  test_text.append(" ".join( [reverse_index.get(i - 3, "#") for i in x_train[i]] ))
print(len(train_text),len(test_text))

1654784/1641221 [==============================] - 0s 0us/step
25000 25000


In [ ]:
print(train_text[0])

# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

**Data Preparation**

Now it's time to prepare our data. 

As we know, each review consists of different number of words. Some reviews could even be one word long. e.g. "nice"

Deep learning models look best when all of the data is in a similar shape. 

Here we consider maximum length of our input sequence to be 100. pad_sequences will add 0's to any reviews which don't have a length of 100.

For example, our one word review above would become: "index(nice) 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0... 99 times"

The same goes for any reviews longer than 100 characters, they will be shortened to a maximum of 100.

In [ ]:
max_review_length = 100
x_train = sequence.pad_sequences(x_train, maxlen=max_review_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_review_length)

In [ ]:
print(x_train[0])

[1415   33    6   22   12  215   28   77   52    5   14  407   16   82
    2    8    4  107  117 5952   15  256    4    2    7 3766    5  723
   36   71   43  530  476   26  400  317   46    7    4    2 1029   13
  104   88    4  381   15  297   98   32 2071   56   26  141    6  194
 7486   18    4  226   22   21  134  476   26  480    5  144   30 5535
   18   51   36   28  224   92   25  104    4  226   65   16   38 1334
   88   12   16  283    5   16 4472  113  103   32   15   16 5345   19
  178   32]


**BUILDING AND TRAINING THE MODEL**

Now our data is ready for some modelling!

Deep learning models have layers.

The top layer takes in the data we've just prepared, the middle layers do some math on this data and the final layer produces an output we can hopefully make use of.

In our case, our model has three layers, 

1. Embedding layer
2. LSTM layer
3. Dense layer.

Our model begins with the line model = Sequential(). Think of this as simply stating "our model will flow from input to output layer in a sequential manner" or "our model goes one step at a time".

**Embedding layer**

The Embedding layer creates a database of the relationships between words.

model.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length)) is saying: add an Embedding layer to our model and use it to turn each of our words into 32 dimensional vector which have some mathematical relationship to each other.

So each of our words will become vectors of dimension 32.

For example, vector of "the" = [0.556433, 0.223122, 0.789654....].

Don't worry for now how this is computed, Keras does it for us.

**LSTM layer**

model.add(LSTM(128)) is saying: add a LSTM layer after our embedding layer in our model and give it 128 units.

LSTM = Long short-term memory. Think of LSTM's as a tap, a tap whichs decides which words flow through the model and which words don't. This layer uses 100 taps to decide which words matter the most in each review.

**Dense layer**

model.add(Dense(1, activation='sigmoid')) is saying: add a Dense layer to the end of our model and use a sigmoid activation function to produce a meaningful output.

A dense layer is also known as a fully-connected layer. This layer connects the 128 LSTM units in the previous layer to 1 unit. This last unit them takes all this information and runs it through a sigmoid function.

Essentially, the sigmoid function will decide if the information should be given a 1 or a -1. 1 for positive and -1 for negative. This is will decided on based on the information passed through by the LSTM layer.


Lastly, we let Keras print a summary of the model we have just built.

In [ ]:
# Define how long the embedding vector will be
embedding_vector_length = 32

# Define the layers in the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=vocab_size))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

print("Model created.")

Model created.


**Compiling the model**

Now we compile our model, which is nothing but configuring the model for training. We use the “adam” optimizer, an algorithm that changes the weights and biases during training. We also choose binary-crossentropy as loss (because we deal with binary classification) and accuracy as our evaluation metric.

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print("Model compiled, ready to be fit to the training data.")

Model compiled, ready to be fit to the training data.


**Summarize the model**

Making a summary of the model will give us an idea of what's happening at each layer.

In the embedding layer, each of our words is being turned into a vector of dimension 32. Because there are 10000 words (max_words), there are 320,000 parameters (32 x 10000).

Parameters are individual pieces of information. The goal of the model is to take a large number of parameters and reduce them down to something we can understand and make use of (less parameters).

The LSTM layer reduces the number of parameters to 82432 = 4 × [128(128+32) + 128].

The final dense layer connects each of the outputs of the LSTM units into one cell (128 + 1).

In [ ]:
# Summarize the different layers in the model
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10000, 32)         320000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 402,561
Trainable params: 402,561
Non-trainable params: 0
_________________________________________________________________
None


**Fitting the model to the training data**

Now our model is compiled, it's ready to be set loose on our training data.

We'll be training for 3 epochs with a batch_size of 64.

Because of our loss and optimzation functions, the model accuracy should improve after each cycle.

model.fit(X_train, y_train, epochs=3, batch_size=64) is saying: fit the model we've built on the training dataset for 3 cycles and go over 64 reviews at a time.

Feel free to change the number of epochs (more cycles) or batch_size (more or less information each step) to see how the accuracy changes.

This will take a few minutes.

In [ ]:
# Fit the model to the training data
results = model.fit(x_train, y_train, epochs=3, batch_size=64,validation_data=(x_test, y_test))

Epoch 1/3
391/391 [==============================] - 16s 42ms/step - loss: 0.4937 - accuracy: 0.7546 - val_loss: 0.3469 - val_accuracy: 0.8486
Epoch 2/3
391/391 [==============================] - 16s 40ms/step - loss: 0.2862 - accuracy: 0.8863 - val_loss: 0.3491 - val_accuracy: 0.8496
Epoch 3/3
391/391 [==============================] - 16s 41ms/step - loss: 0.2335 - accuracy: 0.9098 - val_loss: 0.3818 - val_accuracy: 0.8430


It is time to evaluate our model:

In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=64)
print('Test score:', score)
print('Test accuracy:', acc)

391/391 [==============================] - 2s 4ms/step - loss: 0.3620 - accuracy: 0.8440
Test score: 0.3619990646839142
Test accuracy: 0.8440399765968323


## Below we have some more sample architectures you can try !!

**Extensions**

Let us use LSTM variants. We use check the accuracy by replacing LSTM cell with GRU cell.

In [ ]:
# Define the layers in the model
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_review_length))
#model.add(LSTM(128))
model.add(GRU(32,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

#num_params_layer 3 × [h(h+i) + h]  = 3 × [32(32+64) + 32] = 9312

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           640000    
_________________________________________________________________
gru (GRU)                    (None, 32)                9408      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 649,441
Trainable params: 649,441
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
results = model.fit(x_train, y_train, epochs=3, batch_size=64,validation_data=(x_test, y_test))

Epoch 1/3
391/391 [==============================] - 148s 380ms/step - loss: 0.4708 - accuracy: 0.7598 - val_loss: 0.3517 - val_accuracy: 0.8446
Epoch 2/3
391/391 [==============================] - 148s 379ms/step - loss: 0.2873 - accuracy: 0.8838 - val_loss: 0.3545 - val_accuracy: 0.8466
Epoch 3/3
391/391 [==============================] - 148s 378ms/step - loss: 0.2207 - accuracy: 0.9155 - val_loss: 0.4140 - val_accuracy: 0.8373


In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=64)
print('Test score:', score)
print('Test accuracy:', acc)

391/391 [==============================] - 11s 28ms/step - loss: 0.4140 - accuracy: 0.8373
Test score: 0.4140080511569977
Test accuracy: 0.8372799754142761


**Using LSTM stack layers**

In [ ]:
model= Sequential()
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(units=16, return_sequences=True))
model.add(LSTM(units=8, return_sequences=True))
model.add(LSTM(units=4))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 32)           320000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 16)           3136      
_________________________________________________________________
lstm_3 (LSTM)                (None, 100, 8)            800       
_________________________________________________________________
lstm_4 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 324,149
Trainable params: 324,149
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
results = model.fit(x_train, y_train, epochs=3, batch_size=64,validation_data=(x_test, y_test))

Epoch 1/3
391/391 [==============================] - 12s 31ms/step - loss: 0.4672 - accuracy: 0.7748 - val_loss: 0.3713 - val_accuracy: 0.8397
Epoch 2/3
391/391 [==============================] - 11s 28ms/step - loss: 0.2868 - accuracy: 0.8884 - val_loss: 0.3722 - val_accuracy: 0.8406
Epoch 3/3
391/391 [==============================] - 11s 28ms/step - loss: 0.2153 - accuracy: 0.9215 - val_loss: 0.3897 - val_accuracy: 0.8399


In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=64)
print('Test score:', score)
print('Test accuracy:', acc)

391/391 [==============================] - 3s 9ms/step - loss: 0.3897 - accuracy: 0.8399
Test score: 0.38971391320228577
Test accuracy: 0.8399199843406677


**Using Simple RNN**

In [ ]:
# Define the layers in the model
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_review_length))
#model.add(LSTM(128))
model.add(SimpleRNN(32,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 64)           640000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                3104      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 643,137
Trainable params: 643,137
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
# Fit the model to the training data
results = model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/3
391/391 [==============================] - 55s 140ms/step - loss: 0.6708 - accuracy: 0.5690 - val_loss: 0.5691 - val_accuracy: 0.7017
Epoch 2/3
391/391 [==============================] - 54s 138ms/step - loss: 0.5145 - accuracy: 0.7502 - val_loss: 0.5300 - val_accuracy: 0.7646
Epoch 3/3
391/391 [==============================] - 54s 138ms/step - loss: 0.4577 - accuracy: 0.7926 - val_loss: 0.8073 - val_accuracy: 0.6813


In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=64)
print('Test score:', score)
print('Test accuracy:', acc)

391/391 [==============================] - 4s 9ms/step - loss: 0.8073 - accuracy: 0.6813
Test score: 0.8072788119316101
Test accuracy: 0.6812800168991089
